# Azure Data Explorer Kusto Query Queue: Send

In [ ]:
from azure.storage.queue import QueueClient
from typing import List, Dict, Generator

import pandas as pd
import logging 
import json
import time
import os

logging_timestamp = str(pd.Timestamp.today())
logging_timestamp = logging_timestamp.replace("-","").replace(":","").replace(".","").replace(" ","")
logging.basicConfig(
    filename=f"kqq-send-{logging_timestamp}.log",
    format='%(asctime)s %(levelname)s %(message)s',
    filemode='w'
)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def get_datetime_ranges(
    start_datetime: str, 
    end_datetime: str, 
    frequency: str = '1h'
) -> List[Dict]:
    """
    Converts start and end dates to time range pairs by frequency
    Args:
        start_datetime: time range start, example: "2007-01-01 00:00:00"
        end_datetime: time range end, example: "2007-01-02 00:00:00"
        frequency: time amount between sub-queries, ie. "1h" is 1 hour
    Returns:
        list of dictionaries with start and end timestamps 
    """
    time_ranges = pd.date_range(
        start_datetime, 
        end_datetime, 
        freq=frequency
    )
    start_and_end_datetimes = []
    for each_index, each_time in enumerate(time_ranges):
        if each_index < len(time_ranges)-1:
            start_and_end_datetimes.append(
                {
                    "start_datetime" : str(each_time),
                    "end_datetime" : str(time_ranges[each_index+1])
                }
            )
    return start_and_end_datetimes

def get_query_template(query_template_filename: str) -> str:
    """
    Reads in query template from filename
    Args:
        query_template_filename: path to filename
    Returns:
        kql query template string
    """
    with open(query_template_filename, 'r') as f:
        query_template = f.read()
    return query_template

def generate_each_query_from_template(
    start_datetime:str, 
    end_datetime:str, 
    query_template:str
) -> str:
    """
    Generates query using template and provided datetimes
    Args:
        start_datetime: time range start, example: "2007-01-01 00:00:00"
        end_date: time range end, example: "2007-01-02 00:00:00"
        query_template: kql query template string
    Returns:
        query text string
    """
    query = query_template.replace("<START_DATETIME_STRING>", start_datetime)
    query = query.replace("<END_DATETIME_STRING>", end_datetime)
    return query 

def generate_queries(
    start_datetime: str, 
    end_datetime: str, 
    frequency: str, 
    query_template_filename: str
) -> Generator:
    """
    Generates
    Args:
        start_datetime: time range start, example: "2007-01-01 00:00:00"
        end_date: time range end, example: "2007-01-02 00:00:00"
        frequency: time amount between sub-queries, ie. "1h" is 1 hour
        query_template_filename: path to filename
    Returns:
        generator of dictionaries
    """
    # gets list of datetime ranges 
    datetime_ranges = get_datetime_ranges(
        start_datetime, 
        end_datetime, 
        frequency
    )
    # gets kql query template
    query_template = get_query_template(query_template_filename)
    # generates queries 
    for each_datetime_range in datetime_ranges:
        each_datetime_start = each_datetime_range["start_datetime"]
        each_datetime_end = each_datetime_range["end_datetime"]
        each_query = generate_each_query_from_template(
            each_datetime_start, 
            each_datetime_end, 
            query_template
        )
        each_output = {
            "start_datetime" : each_datetime_start,
            "end_datetime" : each_datetime_end,
            "query" : each_query
        }
        yield json.dumps(each_output)

def create_queue_and_send_messages(
    queue_client: QueueClient, 
    start_datetime: str, 
    end_datetime: str, 
    query_template_filename: str,
    frequency: str = "1h",
    request_wait_seconds: int = 1,
) -> str:
    """
    Args:
        queue_client: azure storage QueueClient object
        start_datetime: time range start, example: "2007-01-01 00:00:00"
        end_datetime: time range end, example: "2007-01-02 00:00:00"
        query_template_filename: path filename to kql query template
        frequency: time amount between sub-queries, ie. "1h" is 1 hour
        request_wait_seconds: sleep time between http requests 
    """
    # add queries to queue
    queries = generate_queries(
        start_datetime, 
        end_datetime, 
        frequency, 
        query_template_filename
    )
    for each_query in queries:
        try:
            queue_client.send_message(each_query)
        except Exception as e:
            print(f"unable to submit query: {each_query}, exception: {e}")
        finally:
            time.sleep(request_wait_seconds)
    queue_properties = queue_client.get_queue_properties()
    logger.info(f"{queue_properties}")
    print(f"{queue_properties}")

## Azure Authentication

In [ ]:
# storage account authentication for queue
# how to: storage account -> access keys -> connection string
# os.environ["AZURE_STORAGE_CONNECTION_STRING"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
storage_connection_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [ ]:
# queue connection 
queue_name = f"kusto-query-queue-123"
queue_client = QueueClient.from_connection_string(
    storage_connection_str, 
    queue_name
)
queue_client.create_queue()

## User Inputs

In [ ]:
start_datetime = "2007-01-01 00:00:00"
end_datetime = "2007-01-01 10:00:00"
query_template_filename = "./azue-data-explorer-kusto-queue-query-template.kql"

## Run

In [ ]:
create_queue_and_send_messages(
    queue_client, 
    start_datetime, 
    end_datetime, 
    query_template_filename,
)